In [68]:
import pyodbc
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import msal
from dotenv import load_dotenv
import os
import requests

In [69]:
"""------------- Crear conecxion con base datos ODBC --------------"""
# Load enviroment variables from the .env file
load_dotenv('enviroment.env')
# access the environment variables
dsn_name  = os.getenv("dsn_name")
user= os.getenv("user")
password = os.getenv("password")
# Cadena de conexión ODBC
conn_str = f"DSN={dsn_name};UID={user};PWD={password}"
# Establecer la conexión
conn = pyodbc.connect(conn_str)
# Crear un cursor
cursor = conn.cursor()


"""--------- datos para el email ---------- """
# Configuración
email_address = 'diidierstev@gmail.com'
email_password = 'bmdq yocj odxt mdxk'

In [70]:
""" ------------------------ Generar token autenticacion con API graph microsoft ---------"""

client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")
tenant_id = os.getenv("tenant_id")

authority = f"https://login.microsoftonline.com/{tenant_id}"
app = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)

scope = ["https://graph.microsoft.com/.default"]
result = app.acquire_token_for_client(scopes=scope)

if "access_token" in result:
    token = result['access_token']
    print(f'Token: {token}')
else:
    print("Error acquiring token:", result.get("error"), result.get("error_description"))

Token: eyJ0eXAiOiJKV1QiLCJub25jZSI6IjdxV0ZpZlhNamtlNkVHOVBMMHR2RlR6czE3cjVER1ZaTjgtU1dWei1tQTgiLCJhbGciOiJSUzI1NiIsIng1dCI6Ik1HTHFqOThWTkxvWGFGZnBKQ0JwZ0I0SmFLcyIsImtpZCI6Ik1HTHFqOThWTkxvWGFGZnBKQ0JwZ0I0SmFLcyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC81NzlkOTgwNi00NDEzLTQ1MGQtODk2Mi0zZDQ2NmZiNzlmZGQvIiwiaWF0IjoxNzE5NDEyMjQzLCJuYmYiOjE3MTk0MTIyNDMsImV4cCI6MTcxOTQxNjE0MywiYWlvIjoiRTJkZ1lQZzE0M2VybzJ4bzc4VTNpdzVQYk05aEJBQT0iLCJhcHBfZGlzcGxheW5hbWUiOiJtdW5kaW1vdG9zX3BxciIsImFwcGlkIjoiYzIxY2IyNGUtMWRhZS00M2RjLTkyMWEtZjJmOTZjMDg3YzcxIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNTc5ZDk4MDYtNDQxMy00NTBkLTg5NjItM2Q0NjZmYjc5ZmRkLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiNzE2ODdkZWQtYzkyMy00NDExLTk2YmEtYTJlMmFiZjU0OTI4IiwicmgiOiIwLkFUUUFCcGlkVnhORURVV0pZajFHYjdlZjNRTUFBQUFBQUFBQXdBQUFBQUFBQUFBMEFBQS4iLCJyb2xlcyI6WyJGaWxlcy5SZWFkV3JpdGUuQXBwRm9sZGVyIiwiQnJvd3NlclNpdGVMaXN0cy5SZWFkLkFsbCIsIlNpdGVzLlNlbGVjdGVkIiwiRmlsZXMuU2VsZWN0ZWRPcGVyYXRp

In [73]:
# Realizar una solicitud

headers = {
    'Authorization': f'Bearer {token}'
}

response = requests.get(f"https://graph.microsoft.com/v1.0/sites/12750d54-6748-4206-97bd-94929c8d55a6/lists/55bf5444-d5f7-412a-a520-a6e70e36bfbb/items", headers=headers)

for form in response.json()['value']:
    response_id = requests.get(f"https://graph.microsoft.com/v1.0/sites/12750d54-6748-4206-97bd-94929c8d55a6/lists/55bf5444-d5f7-412a-a520-a6e70e36bfbb/items/{form['id']}", headers=headers)
    
    fields=response_id.json()['fields']
    pqrs=(fields['Solicitud'],fields['nombre_completo'],fields['tipo_documento'],int(fields['numero_documento']),fields['correo_electronico'],int(fields['telefono_contacto']),fields['fecha_compra'],fields['numero_factura'],fields['area'],fields['descripcion'],fields['sede'],fields['politica_datos'],fields['id'])
    print(pqrs)
    try:
        # Eliminar espacion vacios
        if None in pqrs:
            pqrs=['' if item is None else item for item in list(pqrs)]
            pqrs=tuple(pqrs)
        
        Solicitud,Nombre_completo,Tipo_documento,Documento,Correo,Telefono_contacto,Fecha_compra,No_factura,Area_pqrs,Descripcion_pqrs,Sede,Politica_datos,RecordID=pqrs
        try:
            # Convertir el string a objeto datetime
            fecha_obj = datetime.fromisoformat(Fecha_compra.replace('Z', '+00:00'))
            Fecha_compra=str(fecha_obj.day)+"/"+str(fecha_obj.month)+"/"+str(fecha_obj.year)
        except:
            print(Fecha_compra)
        print(Descripcion_pqrs,Sede)
        """ -------------------- Insertar datos -------------------------- """
        query="""INSERT INTO PQRS (Solicitud,Nombre_completo,Tipo_documento,Documento,Correo,Telefono_contacto,Fecha_compra_texto,No_factura,Area_pqrs,Descripcion_pqrs,Sede,Politica_datos,RecordID) 
        values ('{0}','{1}','{2}',{3},'{4}',{5},'{6}','{7}','{8}','{9}','{10}','{11}',{12})""".format(Solicitud,Nombre_completo,Tipo_documento,Documento,Correo,Telefono_contacto,Fecha_compra,No_factura,Area_pqrs,Descripcion_pqrs,Sede,Politica_datos,RecordID)
        cursor.execute(query)
        conn.commit()
        """----------------------------------------------------------------"""

        """---------------------- Eliminar datos de lista -----------------"""
        #response_id = requests.delete(f"https://graph.microsoft.com/v1.0/sites/12750d54-6748-4206-97bd-94929c8d55a6/lists/55bf5444-d5f7-412a-a520-a6e70e36bfbb/items/{int(RecordID)}", headers=headers)
        """----------------------------------------------------------------"""


        """-------------- Enviar correo ---------"""
        # Crear el mensaje
        mensaje = MIMEMultipart()
        mensaje['From'] = email_address
        mensaje['To'] = Correo
        mensaje['Subject'] = 'Gestion PQRS'

        # Contenido HTML del mensaje
        contenido_html = f"""
            <h2 style="text-align: center">GESTION PQRS</h2>
            <p> Hola, ¡esperamos te encuentres muy bien!
                
            Para Mundimotos es muy importante conocer tu opinión, por esto te confirmamos que hemos recibido tu PQR y la misma quedó radicada con el número {RecordID},  el cual te permitirá hacer seguimiento a tu trámite.
                
            Recuerda que contamos con 15 días hábiles (lunes a viernes) para dar respuesta, de acuerdo con ley 1755 de 2015.
                
            Gracias por escribirnos.</p>
         """

        # Adjuntar el contenido HTML al mensaje
        mensaje.attach(MIMEText(contenido_html, 'html'))

        # Iniciar sesión en el servidor SMTP de Gmail
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as servidor_smtp:
            servidor_smtp.login(email_address, email_password)

            # Enviar el correo electrónico
            servidor_smtp.send_message(mensaje)

    except:
        print('Error')




('Petición', 'WALTER', 'Cédula ciudadanía', 1015427317, 'WHERNANDEZ@CELERIX.COM', 234, '2024-05-28T05:00:00Z', '3245', 'Garantia', 'Bogota', 'T3WHRTE', 'Acepto', '11')
Bogota T3WHRTE
('Queja', 'WALTER', 'Cédula ciudadanía', 1234, 'WHERNANDEZ@CELERIX.COM', 12, '2024-06-11T05:00:00Z', '765', 'Garantia', 'Bogota', 'TEST', 'Acepto', '12')
Bogota TEST


In [72]:
for pqrs in pqrs_tabla_auxiliar:
    try:
        if None in pqrs:
            pqrs=['' if item is None else item for item in list(pqrs)]
            pqrs=tuple(pqrs)

        Solicitud,Nombre_completo,Tipo_documento,Documento,Correo,Telefono_contacto,Fecha_compra,No_factura,Area_pqrs,Descripcion_pqrs,Sede,Politica_datos,RecordID=pqrs
        try:
            Fecha_compra=str(Fecha_compra.strftime('%d/%m/%Y'))
        except:
            print(Fecha_compra)
            #Fecha_compra=''
        #print(Solicitud,Nombre_completo,Tipo_documento,Documento,Correo,Telefono_contacto,Fecha_compra,No_factura,Area_pqrs,Descripcion_pqrs,Sede,Politica_datos,id)
        query="""INSERT INTO PQRS (Solicitud,Nombre_completo,Tipo_documento,Documento,Correo,Telefono_contacto,Fecha_compra_texto,No_factura,Area_pqrs,Descripcion_pqrs,Sede,Politica_datos,RecordID) 
        values ('{0}','{1}','{2}',{3},'{4}',{5},'{6}','{7}','{8}','{9}','{10}','{11}',{12})""".format(Solicitud,Nombre_completo,Tipo_documento,Documento,Correo,Telefono_contacto,Fecha_compra,No_factura,Area_pqrs,Descripcion_pqrs,Sede,Politica_datos,RecordID)
        cursor.execute(query)
        conn.commit()
        query_delete="""Delete from pqrs_mundimoto where RecordID = {0}""".format(int(RecordID))
        #cursor_aux.execute(query_delete)
        #conn_aux.commit()
        

    except:
        print("error")



#cursor_aux.execute("""Delete from pqrs_mundimoto""")
#conn_aux.commit()
#conn_aux.close()
conn.close()

NameError: name 'pqrs_tabla_auxiliar' is not defined